
# Pair Programming Random Forest

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Random Forest en este tipo de problemas. Los objetivos de este pair programming :

In [5]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
from tqdm import tqdm
# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [6]:
#abrimos el DF:
df = pd.read_csv('datos/EST_Social_Network_Ads_Aj_int.csv', index_col=0)
df.head()

,age,estimatedsalary,purchased,gender_Female,gender_Male
0,19,19000,0,0,1
1,35,20000,0,0,1
2,26,43000,0,1,0
3,27,57000,0,1,0
4,19,76000,0,0,1



- ### Ajustad un modelo de Random Forest a nuestros datos. 


Iniciamos nuestro modelo Random Forest

In [7]:
#Separamos los datos en X e y 
X = df.drop('purchased', axis=1)
y = df['purchased']

In [8]:
#generamos los df de entrenamiento y test
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, random_state = 42)


- ### Calculad las métricas a nuestro nuevo modelo. 


In [9]:
#establecer los parámetros
param_rf = {'max_depth': [2, 4, 6, 7], #bajamos la profundidad
         'max_features': [1, 2, 3], #a partir del resultado del R cuadrado.
         'min_samples_split': [50, 100, 150, 200],
         'min_samples_leaf': [30, 70, 150]}

In [10]:
# iniciamos el modelo con GridSearch con los parámetros establecidos previamente
gs_rf = GridSearchCV(
            estimator=RandomForestClassifier(random_state= 42), # aplicación del modelo de DTree
            param_grid= param_rf, # para que testee nuestros hiperparámetros
            cv=10) # crossvalidation 

In [11]:
#ajustamos en el modelo los datos de entrenamiento
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [2, 4, 6, 7], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [30, 70, 150],
                         'min_samples_split': [50, 100, 150, 200]})

In [12]:
#extraemos el mejor modelo:
bosque = gs_rf.best_estimator_
bosque

RandomForestClassifier(max_depth=6, max_features=3, min_samples_leaf=30,
                       min_samples_split=50, random_state=42)

Vemos que, según el GridSearch, el mejor modelo **es exactamente igual que el mejor modelo del decision tree** tiene las siguientes características:
- *max_depth* o profundidad máxima de 2.

- *max_features* o raíz cuadrada de 1.

- *min_samples_leaf* o número mínimo de observaciones que debe de tener cada uno de los nodos hijos. En nuestro caso, 80.

- *min_samples_split* o número mínimo de observaciones que puede tener un nodo para dividirse. En nuestro caso, nuestro modelo necesita 100 observaciones como mínimo para poder dividirse.



In [13]:
# predecimos el valor de la VR según el mejor modelo:

y_best_pred_test = bosque.predict(x_test)
y_best_pred_train = bosque.predict(x_train)

In [14]:
#invocamos la función para extraer las métricas del test y del train:

def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    # para el df test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)
    
    # para el df train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train],
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train],
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    df["modelo"] = modelo
    return df

In [15]:
#aplicamos la función
resultados_rf = metricas(y_test, y_best_pred_test, y_train, y_best_pred_train, 'Random Forest EST')
resultados_rf

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.875000,0.877193,0.793651,0.833333,0.733733,test,Random Forest EST
1,0.885938,0.847222,0.820628,0.833713,0.746945,train,Random Forest EST


- ### Comparad las métricas con los modelos hechos hasta ahora. ¿Cuál es mejor?

In [16]:
#abrimos un csv con los resultados de las métricas de modelos anteriores de RL:
resultados_dt = pd.read_csv('datos/ajustes-modelo-DT.csv', index_col=0)
resultados_dt

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.600000,0.476190,0.158730,0.238095,0.051325,test,R. log ESTANDARIZADA
1,0.682813,0.631579,0.215247,0.321070,0.174923,train,R. log ESTANDARIZADA
0,0.650000,0.000000,0.000000,0.000000,0.000000,test,R. log SIN estandarizar
1,0.640625,0.000000,0.000000,0.000000,0.000000,train,R. log SIN estandarizar
0,0.868750,0.875000,0.777778,0.823529,0.719626,test,Decision Tree EST BM
1,0.885938,0.844037,0.825112,0.834467,0.747476,train,Decision Tree EST BM
0,0.843750,0.839286,0.746032,0.789916,0.666222,test,Decision Tree RLog EST
1,0.928125,0.904110,0.887892,0.895928,0.841042,train,Decision Tree RLog EST


In [17]:
resultados_dt_rf = pd.concat([resultados_dt, resultados_rf], axis=0)
resultados_dt_rf

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.600000,0.476190,0.158730,0.238095,0.051325,test,R. log ESTANDARIZADA
1,0.682813,0.631579,0.215247,0.321070,0.174923,train,R. log ESTANDARIZADA
0,0.650000,0.000000,0.000000,0.000000,0.000000,test,R. log SIN estandarizar
1,0.640625,0.000000,0.000000,0.000000,0.000000,train,R. log SIN estandarizar
0,0.868750,0.875000,0.777778,0.823529,0.719626,test,Decision Tree EST BM
1,0.885938,0.844037,0.825112,0.834467,0.747476,train,Decision Tree EST BM
0,0.843750,0.839286,0.746032,0.789916,0.666222,test,Decision Tree RLog EST
1,0.928125,0.904110,0.887892,0.895928,0.841042,train,Decision Tree RLog EST
0,0.875000,0.877193,0.793651,0.833333,0.733733,test,Random Forest EST
1,0.885938,0.847222,0.820628,0.833713,0.746945,train,Random Forest EST


In [18]:
resultados_dt_rf.to_csv('datos/met-RLo-def.csv')

#### **Interpretación**:


Tras comparar las métricas aplicando los modelos, llegamos a las siguientes conclusiones:

- Partiendo del valor *kappa* de los distintos modelos, los únicos que tiene un nivel de concordania aceptable (> 0.6) y no sufren un overfitting muy remarcable, los modelos que mas se ajustan a una mejor precidición son: Decision Tree EST BM y Ramdon Forest. 

- Dado que nos interesa saber qué factores llevan a una persona a pulsar un botón (casos positivos), prestaremos más atención a los parámetros *precision y recall*.

- Especialmente, nos interesa el valor del parámetro *precision* dado que queremos saber qué % de casos positivos correctos (Verdaderos Positivos) ha sido capaz de predecir el modelo. 

**Por lo tanto, optaremos por usar el modelo Random Forest al tener las mejores métricas**

#### Para explorar el peso de las variables predictoras o independientes:

In [19]:
# vamos a crearnos un dataframe, igual que hicimos en la clase anterior con la importancia de cada una de las variables incluidas en el modelo
importancia_predictores = pd.DataFrame(
                            {'Variables predictoras': x_train.columns,
                             'importancia': bosque.feature_importances_}
                            )

# ordenamos de mayor a menor los resultados
importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados:

round(importancia_predictores, 3)

,Variables predictoras,importancia
0,age,0.612
1,estimatedsalary,0.385
2,gender_Female,0.002
3,gender_Male,0.002


Dado que la variable de género está dividida en dos categorías, vamos a fusionarla en una sola:

In [20]:
#extraemos la relevancia de género, localizándola en el DF:
gender_importancia = importancia_predictores.iloc[[-1, -2]]

#borramos las columnas de género:
importancia_predictores.drop(gender_importancia.index, inplace = True)

#fusionamos y colocamos en tercera posición:
importancia_predictores.loc[2] =  ["gender", gender_importancia["importancia"].sum()
#resultado:
importancia_predictores


,Variables predictoras,importancia
0,age,0.611548
1,estimatedsalary,0.384990
2,gender,0.003461
